In [ ]:
#Install driver

In [3]:
pip install selenium


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
#Import libraries
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import pandas as pd
import regex as re
driver = webdriver.Firefox()

In [5]:
#Libraries for driver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [6]:
#Access the SRA
driver.get('https://www.ncbi.nlm.nih.gov/sra')

In [7]:
#Custome search
# ngs_seq_search = input("Type the name of technology and organism you want to find:")
# elem.send_keys(ngs_seq_search + Keys.RETURN)

#ONT data search
elem = driver.find_element(By.NAME, 'term')
elem.send_keys('direct RNA-seq and bacteria and ONT' + Keys.RETURN)


In [9]:
#Go to run selector
selector = driver.find_element(By.ID, 'id-sra-run-selector-popup')
selector.click()

In [10]:
#Get run selector table
src = driver.page_source
parser = BeautifulSoup(src, "lxml")
selector_table = parser.find("table", attrs={"class": "ui selectable striped celled very compact table single line"})
#selector_table = parser.find("div", attrs={"id": "ph-rs-table"})
headers = selector_table.find_all("th")
headerlist = [h.text.strip() for h in headers [1:]] #Delete spaces and other format

# print(selector_table)

In [11]:
rows = selector_table.findAll('tr')[1:]

In [12]:
#Get rows
sequences_data = [[td.getText().strip() for td in rows[i].findAll('td')[1:]] for i in range(len(rows))]

In [13]:
#Turn table into a df
sequencesdf = pd.DataFrame(sequences_data, columns=headerlist)

In [14]:
#Verify results
sequencesdf

,1Run,2BioProject,3BioSample,4Center Name,5DATASTORE filetype,6Experiment,7Isolation_source,8Library Name,9LibrarySelection,10LibrarySource,...,22Bases,23Bytes,24create_date,25version,26BIOMATERIAL_PROVIDER,27AssemblyName,28type-material,29HOST,30Isolate,31primary_investigator
0,SRR11550236,PRJNA497103,SAMN10247560,UNIVERSITY OF ARKANSAS FOR MEDICAL SCIENCES,"bam, run.zq, sra",SRX8120039,Cultivation,EcoliTypeStrain_dcDNA,RANDOM,TRANSCRIPTOMIC,...,176.75 M,143.86 Mb,2020-04-1720:26:00Z,1,,assembly,neotype strain of Escherichia coli,,,
1,SRR11550247,PRJNA497103,SAMN10247560,UNIVERSITY OF ARKANSAS FOR MEDICAL SCIENCES,"bam, run.zq, sra",SRX8120028,Cultivation,EcoliTypeStrain_dRNA,RANDOM,TRANSCRIPTOMIC,...,237.95 M,184.93 Mb,2020-04-1720:26:00Z,1,,assembly,neotype strain of Escherichia coli,,,
2,SRR13570801,PRJNA575545,SAMN12900250,CENTRE FOR GENOMIC REGULATION,"nanopore, run.zq, sra",SRX9968998,NA,UNM_short,Oligo-dT,SYNTHETIC,...,15.87 M,13.55 Mb,2021-01-2809:22:00Z,1,,,,NA,NA,
3,SRR18061002,PRJNA805453,SAMN25861597,"UNIVERSITY OF MARYLAND, BALTIMORE","fastq, sra",SRX14213007,laboratory,RDMIN20210517,PolyA,TRANSCRIPTOMIC,...,394.95 M,357.82 Mb,2022-02-2405:57:00Z,1,Dr. David Rasko,,,,,
4,SRR18061003,PRJNA805453,SAMN25861595,"UNIVERSITY OF MARYLAND, BALTIMORE","fastq, sra",SRX14213006,laboratory,RDMIN20210507,PolyA,TRANSCRIPTOMIC,...,1.25 G,1.08 Gb,2022-02-2405:48:00Z,1,Dr. David Rasko,,,,,
5,SRR18061004,PRJNA805453,SAMN25861596,"UNIVERSITY OF MARYLAND, BALTIMORE","fastq, sra",SRX14213005,laboratory,RDMIN20201211,PolyA,TRANSCRIPTOMIC,...,2.02 G,1.76 Gb,2022-02-2405:52:00Z,1,Dr. David Rasko,,,,,
6,SRR18061005,PRJNA805453,SAMN25861598,"UNIVERSITY OF MARYLAND, BALTIMORE","fastq, sra",SRX14213004,laboratory,RDMIN20200702,PolyA,TRANSCRIPTOMIC,...,2.21 G,1.91 Gb,2022-02-2405:46:00Z,1,Dr. David Rasko,,,,,
7,SRR18070402,PRJNA805453,SAMN25861597,"UNIVERSITY OF MARYLAND, BALTIMORE",nanopore,SRX14222110,laboratory,RDMIN20210517-fast5,PolyA,TRANSCRIPTOMIC,...,,,,,Dr. David Rasko,,,,,
8,SRR18070403,PRJNA805453,SAMN25861595,"UNIVERSITY OF MARYLAND, BALTIMORE",nanopore,SRX14222109,laboratory,RDMIN20210507-fast5,PolyA,TRANSCRIPTOMIC,...,,,,,Dr. David Rasko,,,,,
9,SRR18070404,PRJNA805453,SAMN25861596,"UNIVERSITY OF MARYLAND, BALTIMORE",nanopore,SRX14222108,laboratory,RDMIN20201211-fast5,PolyA,TRANSCRIPTOMIC,...,,,,,Dr. David Rasko,,,,,


In [15]:
#Pull all run IDs
runid = sequencesdf['1Run'].to_list()
print(runid)

['SRR11550236', 'SRR11550247', 'SRR13570801', 'SRR18061002', 'SRR18061003', 'SRR18061004', 'SRR18061005', 'SRR18070402', 'SRR18070403', 'SRR18070404', 'SRR18070405', 'SRR21654270', 'SRR21654271', 'SRR21654272', 'SRR23886068']


In [16]:
#Pull only run experiments with fast5 files
fast5_df = sequencesdf.loc[sequencesdf['5DATASTORE filetype']=='nanopore']
print(fast5_df)


           1Run  2BioProject    3BioSample  \
7   SRR18070402  PRJNA805453  SAMN25861597   
8   SRR18070403  PRJNA805453  SAMN25861595   
9   SRR18070404  PRJNA805453  SAMN25861596   
10  SRR18070405  PRJNA805453  SAMN25861598   
12  SRR21654271  PRJNA882509  SAMN30941647   

                                         4Center Name 5DATASTORE filetype  \
7                   UNIVERSITY OF MARYLAND, BALTIMORE            nanopore   
8                   UNIVERSITY OF MARYLAND, BALTIMORE            nanopore   
9                   UNIVERSITY OF MARYLAND, BALTIMORE            nanopore   
10                  UNIVERSITY OF MARYLAND, BALTIMORE            nanopore   
12  SIR RUN RUN SHAW HOSPITAL, ZHEJIANG UNIVERSITY...            nanopore   

    6Experiment        7Isolation_source        8Library Name  \
7   SRX14222110               laboratory  RDMIN20210517-fast5   
8   SRX14222109               laboratory  RDMIN20210507-fast5   
9   SRX14222108               laboratory  RDMIN20201211-fast5   


In [17]:
df5 = fast5_df['1Run'].to_list()
print(df5)

['SRR18070402', 'SRR18070403', 'SRR18070404', 'SRR18070405', 'SRR21654271']


In [18]:
#Fast5 accession number for metadata
file = open("an_f5.txt", "w")
for an in df5:
    file.write(an + "\n")
file.close()

In [19]:
#Gather all links to data-access
links =[]
for run in runid:
    sra_data_access = f'https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc={run}&display=data-access'
    separate = re.findall(r'(https?://\S+)', sra_data_access)
    links.append(separate)

In [20]:
print(links)

[['https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc=SRR11550236&display=data-access'], ['https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc=SRR11550247&display=data-access'], ['https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc=SRR13570801&display=data-access'], ['https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc=SRR18061002&display=data-access'], ['https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc=SRR18061003&display=data-access'], ['https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc=SRR18061004&display=data-access'], ['https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc=SRR18061005&display=data-access'], ['https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc=SRR18070402&display=data-access'], ['https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc=SRR18070403&display=data-access'], ['https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc=SRR18070404&display=data-access'], ['https://trace.ncbi.nlm.nih.

In [21]:
#Get only fast5 files
fast5 = []
for run in df5:
    sra_data_access = f'https://trace.ncbi.nlm.nih.gov/Traces/?view=run_browser&acc={run}&display=data-access'
    separate = re.findall(r'(https?://\S+)', sra_data_access)
    datalink = fast5.append(separate)


In [22]:
#CSV file with links to access bucket links.
df1 = pd.DataFrame(fast5)
print(df1)
df1.to_csv('/Users/irazemagarcia/MS_Bioinformatics/Advanced Data Management/Final project/Final_script/data/fast5url.csv',header=None,index=False)


                                                   0
0  https://trace.ncbi.nlm.nih.gov/Traces/?view=ru...
1  https://trace.ncbi.nlm.nih.gov/Traces/?view=ru...
2  https://trace.ncbi.nlm.nih.gov/Traces/?view=ru...
3  https://trace.ncbi.nlm.nih.gov/Traces/?view=ru...
4  https://trace.ncbi.nlm.nih.gov/Traces/?view=ru...
